In [ ]:

import os
import faiss
import json
import glob
import time

from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_community.docstore import InMemoryDocstore
from langchain.schema import Document
from uuid import uuid4
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

from collections import deque


In [ ]:
os.environ["OpenAI_API_KEY"] = os.getenv("GPT_API")

model = ChatOpenAI(model ="gpt-4o")

In [ ]:
# folder_path = "food_recipe/"

# # 병합된 데이터를 저장할 리스트
# merged_data = []

# # 폴더 내 모든 JSON 파일 읽기
# for file_name in os.listdir(folder_path):
#     if file_name.endswith(".json"):  # JSON 파일만 처리
#         file_path = os.path.join(folder_path, file_name)
#         with open(file_path, 'r', encoding='utf-8') as file:
#             try:
#                 # JSON 데이터 불러오기
#                 data = json.load(file)
#                 merged_data.append(data)  # 병합 리스트에 추가
#                 print(f"Added data from {file_name}")
#             except json.JSONDecodeError as e:
#                 print(f"Error decoding {file_name}: {e}")

# # 병합된 데이터를 하나의 JSON 파일로 저장
# output_path = os.path.join(folder_path, "final_recipes_data.json")
# with open(output_path, 'w', encoding='utf-8') as output_file:
#     json.dump(merged_data, output_file, indent=4, ensure_ascii=False)

# print(f"병합된 JSON 데이터가 {output_path}에 저장되었습니다.")


In [ ]:
# with open("food_recipe/food_recipes_1-68.json",'r',encoding='utf-8') as file:
#     data = json.load(file)

In [ ]:
# document_recipe = [
#     Document(page_content=str(recipe), metadata={"요리명": recipe["요리명"],"요리재료" : recipe["요리재료"], "기본정보" : recipe["기본정보"], "조리순서" : recipe["조리순서"]})
#     for recipe in data
# ]

In [ ]:
# document_recipe

In [ ]:
embeddings = OpenAIEmbeddings(model = "text-embedding-3-small")

In [ ]:
# index  = faiss.IndexFlatL2(len(embeddings.embed_query("레시피")))

# recipe_store = FAISS(
#     embedding_function=embeddings,
#     index=index,
#     docstore=InMemoryDocstore(),
#     index_to_docstore_id={}
# )

In [ ]:
# uuids = [str(uuid4()) for _ in range(len(document_recipe))]

# recipe_store.add_documents(documents=document_recipe, ids = uuids)

In [ ]:
# recipe_store.save_local("./food_db")
recipes = FAISS.load_local("./food_db." ,embeddings, allow_dangerous_deserialization=True)

In [ ]:
retriever = recipes.as_retriever(search_type="similarity", search_kwargs={"k": 5})

In [ ]:
# prompt = ChatPromptTemplate.from_messages([
#     ("system", "너는 주어진 데이터로만 답변을 할 수 있어."),
#     ("system", "너는 요리의 전문가야."),
    
    
#     ("system", "질문에 답변할 때는 항상 데이터를 세밀히 학습하고 정확한 답변을 생성해야 해."),
#     ("system", "만약 데이터 안에 유저의 질문이 없다면, 양해를 구하고 관련 데이터를 제공할 수 없음을 알리도록 해."),
#     ("system", "질문에 계산이 필요한 경우, 데이터를 기반으로 정확하게 계산하여 결과를 제공해야 합니다."),
#     ("system", "질문에 대한 답변을 생성 하기전에 검증을 마친 후에 생성해줘."),
#     ("system", "부족한 부분이 있다고 생각하면, 다른 데이터를 참조해줘"),

    
#     ("system", "다음은 답변 형식의 예시야: \n"
#                "user: 미역국을 만들고 싶어.\n"
#                "ai: 미역국의 재료는 ~입니다.\n"
#                "ai: 미역국을 만드는 순서는 다음과 같습니다:\n"
#                "ai: 첫 번째 ~~~, 두 번째 ~~~입니다.\n"),
    
    
#     ("system", "또 다른 예시를 들어줄게:\n"
#                "user: 내가 만들려고 하는 미역국의 칼로리는 얼마야?\n"
#                "ai: 미역국의 칼로리는 1인분당 ~kcal입니다. 이 레시피는 ~인분 기준이므로, 총 칼로리는 ~kcal입니다.\n"),
    
    
#     ("system", "추가 예시:\n"
#                "user: 5인분의 레시피로 수정해줘.\n"
#                "ai: 현재 미역국 레시피는 ~인분 기준입니다. 5인분으로 수정된 재료는 다음과 같습니다:\n"
#                "ai: ~~~.\n"
#                "이때도 순서를 전부 알려줘"),
    
    
#     ("system", "추가 예시:\n"
#                "user: 된장찌개 끓이는법을 알려줘.\n"
#                "ai: 된장찌개는 재료에 따라 나뉩니다. 현재 ~ 한 된장찌개 레시피가 있습니다 어떤 된장찌개 레시피를원하십니까?:\n"
#                ),
    
    
#     ("user", "다음과 같은 데이터를 학습해:\n{data}"),
#     ("user", "그리고 질문에 답해:\n{question}")
# ])

In [ ]:
# # 위치 설정
path = "Prompts/"


system_files = ["Require_decide.txt","Food_recipe.txt","Food_recommend.txt"]


system_message = []
for txt in system_files :
    with open(os.path.join(path, txt),"r", encoding="UTF-8") as f:
       
        content = f.read().replace("\\n", "\n")
        system_message.append(("system", content))


system_message.append(("user", "context : {context}\\n\\nQuestion: {question}"))


prompt = ChatPromptTemplate.from_messages(system_message)
prompt 


In [ ]:
contextual_prompt = prompt

In [ ]:
class DebugPassThrough(RunnablePassthrough):
    
    def invoke(self, *args, **kwargs):
        output = super().invoke(*args,**kwargs)
        return output
    
    
class ContextToText(RunnablePassthrough):
    def invoke(self, inputs, config = None, **kwargs):
        return {"data": inputs["data"], "question": inputs["question"]}
    
    
rag_chain_divide = {
    "data" : retriever,
    "question" : DebugPassThrough()


# contextual_prompt에 기능을 분리시키는 프롬프트 입력 (~라면 a를 반환, ~라면 b를 반환)
} | DebugPassThrough() | ContextToText()| contextual_prompt | model 

In [ ]:
# 대화 히스토리를 저장할 공간
chat_history = deque(maxlen = 10)
log = []
timestamp = time.time()
output_file = f"log/output_log_{timestamp}.json"


while True:
    print("-----------------------------")
    
    
    query = input("질문을 입력해 주세요 (break 입력시 종료됩니다) : ")
    
    
    if query.lower() == "break":
        # 결과를 JSON 파일로 저장
        with open(output_file, "w", encoding="utf-8") as f:
            json.dump(log, f, ensure_ascii=False, indent=4)
        break
    
    # 히스토리에 사용자 입력 추가
    chat_history.append({"role": "user", "content" : query})
    
    
    #모델의 입력 구성
    model_input = "\n".join([f"{msg['role']}: {msg['content']}" for msg in chat_history])


    #응답 생성
    response = rag_chain_divide.invoke(model_input)
    
    
    # 히스토리에 모델의 응답 추가
    chat_history.append({"role": "assistant", "content": response.content})
    
    
    #저장
    record = {
        "질문" : query,
        "답변" : response.content,
        "기록" : list(chat_history)
    }
    log.append(record)
    

    print("Question : ", query)
    
    print(response.content)